In [4]:
import pandas as pd
import numpy as np
test =pd.read_csv('test_descriptors.csv')
test.iloc[0] # 
print(type(test))
 
test 

<class 'pandas.core.frame.DataFrame'>


,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,13046,13047,13048,13049,13050,13051,13052,13053,13054,13055
0,0,2.0,10.0,57.0,112.0,43.0,1.0,1.0,4.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,0.0,31.0,73.0,75.0,24.0,5.0,0.0,1.0,12.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,25.0,19.0,43.0,82.0,29.0,8.0,6.0,23.0,63.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,17.0,24.0,2.0,2.0,3.0,1.0,2.0,6.0,15.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,42.0,5.0,1.0,3.0,14.0,2.0,0.0,6.0,133.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5,10.0,0.0,0.0,0.0,0.0,4.0,27.0,30.0,43.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,6,40.0,39.0,6.0,10.0,28.0,1.0,0.0,0.0,13.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,7,16.0,20.0,8.0,3.0,2.0,1.0,6.0,8.0,47.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,8,66.0,1.0,3.0,13.0,21.0,14.0,10.0,44.0,27.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,9,2.0,11.0,3.0,3.0,2.0,5.0,76.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
